In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Size Diameter Rugs_Runners'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=751
attribute_id_input=4629

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r', encoding='utf-8') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A



In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

1139

In [7]:
# def regex_patterns(unit):
#     return fr'(?i)(\d+\.\d+[\W]?[\W]?[\W]{unit}\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]{unit}\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]{unit}\b)|()'
# print(regex_patterns('Le?n?g?t?h?'))
# print(regex_patterns('He?i?g?h?t?'))
# print(regex_patterns('Wi?d?t?h?'))
# print(regex_patterns('Diam?e?t?e?r?'))

# Good Stuff

In [8]:
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [9]:
regex_pattern_diameter=r'(?i)(\d+\")|()'
df['match_rug'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))

In [10]:
height_new=df[(df['match_rug'].astype(str)!='[]')]
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: natsorted(x))
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: re.sub(r"\\?'.?diam?e?t?e?r?"," ft",str(x))).apply(lambda x: re.sub(r'(?i)\\?\".?diam?e?t?e?r?|\\?\”?.?diam?e?t?e?r?|\\\" H',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

curate(height_new, 'matches_diameter', formatted_attribute, 'a-eghj-mo-su-z')
matchesdiam=height_new[['external_id',curation_col]]
rounding(height_new, 'matches_diameter','a-eghj-mo-su-z')

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, matches_diameter, matches_ft, new_matches, blank_matches, blank_matches_long, match_rug, Q:size_diameter_rugs_runners, rounding]
Index: []

In [11]:
df[0:500]

product_id external_id  \
0     817342982    66809609   
1    1469460287   200002582   
2    1469460281   200002576   
3    1469460279   200002575   
4    1469460276   200002572   
5    1469460269   200002565   
6    1469460263   200002559   
7    1469460259   200002556   
8    1469460050   200002368   
9    1469460044   200002363   
10   1469460043   200002362   
11   1469460042   200002361   
12   1469460037   200002357   
13   1469460033   200002353   
14   1469460029   200002349   
15   1469460018   200002339   
16   1469460015   200002336   
17   1469460006   200002328   
18   1469460000   200002322   
19   1469459997   200002319   
20   1469459994   200002316   
21   1469459991   200002314   
22   1469459990   200002313   
23   1469459986   200002309   
24   1469459983   200002307   
25   1469460428   200002702   
26   1469459974   200002299   
27   1469459961   200002287   
28   1469459957   200002284   
29   1469460416   200002692   
30   1469460412   200002689   
31   1469459946   200002273   
32   1469460411   200002688   
33   1469460407   200002684   
34   1469459940   200002268   
35   1469460398   200002677   
36   1469459936   200002265   
37   1469459934   200002263   
38   1469459932   200002261   
39   1469460397   200002676   
40   1469460394   200002673   
41   1469459927   200002256   
42   1469460387   200002667   
43   1469459919   200002248   
44   1469459916   200002245   
45   1469459907   200002237   
46   1469460380   200002661   
47   1469459904   200002234   
48   1469459903   200002233   
49   1469460374   200002656   
50   1469460369   200002652   
51   1469459891   200002222   
52   1469459888   200002219   
53   1469460368   200002651   
54   1469459884   200002216   
55   1469459883   200002215   
56   1469459882   200002214   
57   1469460366   200002649   
58   1469460362   200002646   
59   1469460358   200002642   
60   1469459877   200002210   
61   1469460348   200002635   
62   1469459873   200002206   
63   1469459863   200002197   
64   1469459859   200002194   
65   1469460249   200002547   
66   1469460246   200002544   
67   1469460242   200002540   
68   1469460232   200002531   
69   1469460228   200002527   
70   1469460227   200002526   
71   1469460223   200002522   
72   1469460220   200002520   
73   1469460216   200002516   
74   1469460212   200002512   
75   1469460210   200002510   
76   1469460207   200002508   
77   1469460205   200002506   
78   1469460203   200002504   
79   1469460188   200002491   
80   1469460187   200002490   
81   1469460185   200002488   
82   1469460179   200002483   
83   1469460176   200002480   
84   1469460174   200002479   
85   1469460173   200002478   
86   1469460172   200002477   
87   1469460163   200002469   
88   1469460160   200002467   
89   1469460158   200002465   
90   1469460155   200002462   
91   1469460152   200002460   
92   1469460150   200002458   
93   1469460139   200002448   
94   1469460136   200002445   
95   1469460129   200002439   
96   1469460125   200002435   
97   1469460123   200002433   
98   1469460121   200002431   
99   1469460111   200002423   
100  1469460104   200002417   
101  1469460102   200002415   
102  1469460097   200002412   
103  1469460096   200002411   
104  1469460093   200002408   
105  1469460092   200002407   
106  1469460087   200002403   
107  1469460086   200002402   
108  1469460076   200002393   
109  1469460067   200002384   
110  1469460060   200002377   
111  1469459401   200001787   
112  1469459399   200001785   
113  1469459395   200001782   
114  1469459392   200001779   
115  1469459388   200001776   
116  1469459386   200001774   
117  1469459381   200001770   
118  1469459380   200001769   
119  1469459379   200001768   
120  1469459378   200001767   
121  1469459377   200001766   
122  1469459375   200001765   
123  1469459373   200001763   
124  1469459366   200001756   
125  1469459330   200001727   
126  1469459329   200001726   
127  1469459327   200001724   
128  1

In [12]:
matchesdiam

Empty DataFrame
Columns: [external_id, Q:size_diameter_rugs_runners]
Index: []

# NA

In [13]:
df_na=df[(df['matches_diameter'].astype(str)=='[]')]
print(len(df_na))
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
matchesna

1139


external_id Q:size_diameter_rugs_runners
0       66809609                          n/a
1      200002582                          n/a
2      200002576                          n/a
3      200002575                          n/a
4      200002572                          n/a
...          ...                          ...
1134   200002597                          n/a
1135    69770769                          n/a
1136    69495955                          n/a
1137    64557076                          n/a
1138    16786233                          n/a

[1139 rows x 2 columns]

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matchesna))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesdiam))

matchesna
__


In [15]:
print(len(df))
print(len(matchesna)+len(matchesdiam))

1139
1139


In [16]:
df[df['external_id'].astype(str)=='63198409']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, bucket_id, values, matches_diameter, matches_ft, new_matches, blank_matches, blank_matches_long, match_rug]
Index: []

In [17]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [18]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, matchesna)   

In [19]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, matchesdiam)  